### No anomaly script and extract so/sn 


- specify path will read the filename 

In [152]:
# read the data frame and see the data insight
import pandas as pd
import numpy as np
import sys
df_no = pd.read_csv(r"C:\Users\ijohan\Desktop\analysis\june\31072019\JUNE_LATEST_NO_ANOMALY_DATA_31072019_340PM.csv")
print("Number of row and columns data points:",df_no.shape)
df_no.columns

Number of row and columns data points: (3124, 35)


Index(['Unnamed: 0', 'State', 'State_Description', 'Business_Area___Key',
       'Channel', 'Complaint_ID', 'CA_Number', 'Unit', 'Category',
       'Sub_Category', 'Creation_Date', 'SN_Number', 'SO_Number', 'BPEM_Case',
       'Notes', 'SN_Remarks_31July', 'BPEM_Notes_31July', 'Status_31July',
       'Verified_Root_Cause_Type', 'Verified_Root_Cause', 'Root_Cause_Measure',
       'Refund/Arrears', 'Dt/Cr_Amount', 'MB_Reason', 'No_Anomaly_Root_Cause',
       'Date_Found', 'SO_transfer_SN', 'Refund/Arrears_Outliers?',
       '%_Refund/Bill', 'PORTION', 'MAY_TARIFTYP', 'BILL_DURATION_JUNE',
       'ACT/EST/FIN', 'MAY_CURRENT_CHARGES', 'JUNE_CURRENT_CHARGES'],
      dtype='object')

- select only particular columns 

In [153]:
df_no = df_no[['Complaint_ID',  'Category',
       'Sub_Category', 'SN_Remarks_31July','Verified_Root_Cause']]

<p>
    Let's understand above data. There are 5 fields above: <br>
    <ul>
        <li><b>Complaint_ID : </b></li>
        <li><b>Category : </b></li>
        <li><b>Sub_Category : </b></li>
        <li><b>SN_Remarks_31July :</b></li>
        <li><b>Verified_Root_Cause :</b></li>
    </ul>
    
Keep doing more analysis  on above data.

In [154]:
df_no.head() #select the top5 row for inspection 

Complaint_ID                 Category      Sub_Category  \
0    1000684273  OPC Meter Accuracy Test  High Bill Normal   
1    1000697304  OPC Meter Accuracy Test  High Bill Normal   
2    1000678345  OPC Meter Accuracy Test  High Bill Normal   
3    1000703326  OPC Meter Accuracy Test  High Bill Normal   
4    1000692865  OPC Meter Accuracy Test  High Bill Normal   

                                   SN_Remarks_31July     Verified_Root_Cause  
0   Internal note 07.06.2019 15:12:21 10077213  p...                METER OK  
1   Internal note 13.06.2019 15:46:13 10076230  p...                METER OK  
2   Internal note 03.06.2019 11:14:39 10091193  m...                METER OK  
3   Internal note 17.06.2019 10:39:55 60068218 - ...                METER OK  
4   Internal note 12.06.2019 11:52:38 10088233  m...  WITHIN PREV PEAK RANGE

In [155]:
df_no.info() #showing the dataype for each column 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3124 entries, 0 to 3123
Data columns (total 5 columns):
Complaint_ID           3124 non-null int64
Category               3124 non-null object
Sub_Category           3124 non-null object
SN_Remarks_31July      2900 non-null object
Verified_Root_Cause    3124 non-null object
dtypes: int64(1), object(4)
memory usage: 122.1+ KB


In [156]:
len(df_no) # df count of row 

3124

In [157]:
#changing the columns name with underscore
df_no.columns = df_no.columns.str.replace(' ','_')

In [158]:
#changing the columns name with underscore
df_no.columns = df_no.columns.str.replace('-','_')

In [159]:
# checking for the null values again.
df_no.isnull().sum()

Complaint_ID             0
Category                 0
Sub_Category             0
SN_Remarks_31July      224
Verified_Root_Cause      0
dtype: int64

### Cleansing on sn remarks column

- dont ever try to skip to run all the commands step by step, else u will be in trouble

In [160]:
df_no = df_no[['Complaint_ID',  'Category',
       'Sub_Category', 'SN_Remarks_31July','Verified_Root_Cause']]

#df_sn = df_sn.fillna(" ")

df_no.head()

Complaint_ID                 Category      Sub_Category  \
0    1000684273  OPC Meter Accuracy Test  High Bill Normal   
1    1000697304  OPC Meter Accuracy Test  High Bill Normal   
2    1000678345  OPC Meter Accuracy Test  High Bill Normal   
3    1000703326  OPC Meter Accuracy Test  High Bill Normal   
4    1000692865  OPC Meter Accuracy Test  High Bill Normal   

                                   SN_Remarks_31July     Verified_Root_Cause  
0   Internal note 07.06.2019 15:12:21 10077213  p...                METER OK  
1   Internal note 13.06.2019 15:46:13 10076230  p...                METER OK  
2   Internal note 03.06.2019 11:14:39 10091193  m...                METER OK  
3   Internal note 17.06.2019 10:39:55 60068218 - ...                METER OK  
4   Internal note 12.06.2019 11:52:38 10088233  m...  WITHIN PREV PEAK RANGE

In [161]:
df_no.shape

(3124, 5)

In [162]:
#import all the libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ijohan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ijohan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ijohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ijohan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [163]:
#where the data cleaning begin in sn remarks column
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",str(text))
  text=re.sub("(\\d|\\W)+"," ",str(text))
  text = text.replace("gmail", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.upper()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)


In [164]:
#call the function and specify the sn remarks column
df_no['clean_sn_remarks'] = df_no['SN_Remarks_31July'].apply(clean_txt)


In [165]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_jobid = tfidf_vectorizer.fit_transform((df_no['clean_sn_remarks'])) #fitting and transforming the vector
tfidf_jobid

<3124x9142 sparse matrix of type '<class 'numpy.float64'>'
	with 98968 stored elements in Compressed Sparse Row format>

In [166]:
#generate unigram, bigram, trigram 
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
def get_ngrams(text, n):
    #n = 3
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

df_no['clean_sn_remarks1']  = df_no['clean_sn_remarks'].astype(str).apply(get_ngrams, n=1)
df_no['clean_sn_remarks2']  = df_no['clean_sn_remarks'].astype(str).apply(get_ngrams, n=2)
df_no['clean_sn_remarks3']  = df_no['clean_sn_remarks'].astype(str).apply(get_ngrams, n=3)
#sn_remarks['clean_sn_remark_grams4']  = sn_remarks['clean_sn_remark'].astype(str).apply(get_ngrams, n=4)


In [167]:
df_no['clean_sn_remark_grams'] = df_no['clean_sn_remarks1'] + df_no['clean_sn_remarks2'] +df_no['clean_sn_remarks3']# + sn_remarks['clean_sn_remark_grams4']

In [168]:
del df_no['clean_sn_remarks1']
del df_no['clean_sn_remarks2']
del df_no['clean_sn_remarks3']
#del sn_remarks['clean_sn_remark_grams4']

- u can simply add new keywords just append the new keyword in below set of list.

In [169]:
no_anomaly_customer_consumption_lifestyle_festive_keywords = ['BULAN PUASA','RAMADHAN','RAYA','CUTI RAYA','MUSIM RAYA','MUSIM PERAYAAN','CUTI PERAYAAN','MUSIMRAYA','BUAT KUIH','CUTI PERAYAAN','SEASONAL CONS BEHAVIOUR'] 


In [170]:
df_no['keywords'] = [no_anomaly_customer_consumption_lifestyle_festive_keywords for _ in range(len(df_no))]


In [171]:
# https://stackoverflow.com/questions/42683249/python-pandas-how-to-match-list-of-strings-with-a-dataframe-column
# Define function to search for words in two columns
def search_func(row):
    matches = [test_value in row["clean_sn_remark_grams"]
               for test_value in row["keywords"]]

    if any(matches):
        return 1
    else:
        return 0




In [172]:
df_no['no_anomaly_customer_consumption_lifestyle_festive_keywords_flag'] = df_no.apply(search_func, axis=1)

In [173]:
df_no.no_anomaly_customer_consumption_lifestyle_festive_keywords_flag.value_counts()

0    3124
Name: no_anomaly_customer_consumption_lifestyle_festive_keywords_flag, dtype: int64

- u can simply add new keywords just append the new keyword in below set of list.

In [174]:
smart_meter_keywords = ['SMART METER','TUKAR SMART METER','Bilik Gerakan SM'] 

In [175]:
df_no['keywords'] = [smart_meter_keywords for _ in range(len(df_no))]

In [176]:
df_no['smart_meter_keywords_flag'] = df_no.apply(search_func, axis=1)

In [177]:
df_no.smart_meter_keywords_flag.value_counts()

0    3124
Name: smart_meter_keywords_flag, dtype: int64

- u can simply add new keywords just append the new keyword in below set of list.

In [178]:
no_anomaly_customer_consumption_lifestyle_school_holiday_keywords = ['SCHOOL HOLIDAY','CUTI SEKOLAH','MUSIM CUTI SEKOLAH','CUTI'] 


In [179]:
# Add new key word column for each column
df_no['keywords'] = [no_anomaly_customer_consumption_lifestyle_school_holiday_keywords for _ in range(len(df_no))]

In [180]:
df_no['no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag'] = df_no.apply(search_func, axis=1)

In [181]:
df_no.no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag.value_counts()

0    3124
Name: no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag, dtype: int64

- u can simply add new keywords just append the new keyword in below set of list.

In [182]:
electrical_within_usage_keywords = ['PENGGUNAAN SAMA','KEGUNAAN SAMA','TIADA DI RUMAH','MALAM SAHAJA','TIADA PENAMBAHAN BARANG','TIADA PENAMBAHAN ERALATAN','TIADA PENAMBAHAN PERLATAN','KEGUNAAN MASIHMACAM BIASA','KEGUNAAN MASIHMACAM BIASA','WALAUPUNPENGGUNAAN SAMA' ,'TIADAPENAMBAHAN BARANG','KEGUNAAN ADALAH SAMA','WAKTU MALAM','KEGUNAAN PREMIS ADALAH SAMA','KEGUNAAN ADALAH SAMA','KEGUNAAN ADALAH NORMAL','BACA BETUL','METER BETUL','BACAAN BETUL','METER JITU','BACAAN BIL BETUL','TIDAK MENAMBAH PERALATAN ELEKTRIK','PERALATAN ELEKTRIK SAMA','BIL BETUL','BACAAN BIL BETUL','METER OK','BACAAN OK','KEADAAN BAIK','BERRPUAS HATI','PENGGUNA BERRPUAS HATI','METER BAIK','ADALAH BETUL','PENGGUNAAN SEPERTI BIASA','TIADA PENAMBAHAN','AIR COND','PETI SEJUK','COWAY','PETI AIS','PENAMBAHAN PERALATAN ELEKTRIK','FRIDGE','FREEZER','AIRCOND','WASHING MACHINE','CEILING FAN','REFGIRATOR','LAMP','BASIC HOUSE APPLIANCES','ADA AIRCOND' ,'PERKAKAS BIASA','PERKAKAS BAISA','PENGHAWA DINGIN','BLOWE','OVEN','NASI','DAPUR GAS','READING OK','REDUCTION IN USAGE','WITHIN PREVIOUS PEAK RANGE','METER OK','SHORT BILL PERIOD','FURTHER INVESTIGATION','TIADA MASALAH','BACAAN TEPAT','BACAAN OK','TIADA KESALAHAN BACAAN','TIADA MASALAH PADA BACAAN','METER OK ERROR','METER OK RALAT','BENAR','BIL BETUL','BIL TEPAT','BIL BENAR','PENGGUNAAN SAMA','TIADA PENAMBAHAN PERALATAN','TERATUR','BAIK','BACAAN OK','TIADA MASALAH'] 



In [183]:
# Add new key word column for each column
df_no['keywords'] = [electrical_within_usage_keywords for _ in range(len(df_no))]

In [184]:
df_no['electrical_within_usage_keywords_flag'] = df_no.apply(search_func, axis=1)

In [185]:
df_no.electrical_within_usage_keywords_flag.value_counts()

0    3067
1      57
Name: electrical_within_usage_keywords_flag, dtype: int64

- u can simply add new keywords just append the new keyword in below set of list.

In [186]:
weather_keywords = ['KEMARAU','PANAS','CUACA PANAS','CUACA YANG PANAS']


In [187]:
# Add new key word column for each column
df_no['keywords'] = [weather_keywords for _ in range(len(df_no))]

In [188]:
df_no['weather_keywords_flag'] = df_no.apply(search_func, axis=1)

In [189]:
df_no.weather_keywords_flag.value_counts()


0    3124
Name: weather_keywords_flag, dtype: int64

- u can simply add new keywords just append the new keyword in below set of list.

In [190]:
move_out_keywords = ['PINDAH','PREMIS KOSONG','KOSONG','SUDAH KOSONG','TIADA ORANG','TIADA LOAD','TIADA ORANG DIDUDUKI','PINDAH KELUAR','TIDAK DIDUDUKI','PINDAH','PINDAH KELUAR']


In [191]:
df_no['keywords'] = [move_out_keywords for _ in range(len(df_no))]

In [192]:
df_no['move_out_keywords_flag'] = df_no.apply(search_func, axis=1)

In [193]:
df_no.move_out_keywords_flag.value_counts()

0    3123
1       1
Name: move_out_keywords_flag, dtype: int64

- u can simply add new keywords just append the new keyword in below set of list.

In [194]:
tiada_orang_keywords = ['AWAY FROM HOME','JARANG TINGGAL DI PREMIS','BERCUTI','JARANG TINGGAL DI RUMAH','TIADA ORANG','SEKALI SEKALA','JARANG GUNA ELEKTRIK','TIADA KEGUNAAN','RUMAH TIADA ORANG','MALAM','SIANG TIADA','SIANG TIADA ORANG','SIANG XDE ORANG','SIANG XDE PENGHUNI','SIANG TIADA PENGHUNI','TIADA KEGUNAAN DI SIANG HARI','TIADA KEGUNAAN PADA SIANG HARI','SIANG BEKERJA','MALAM ADA','SIANG HARI','MALAM SAJA ADA','PETANG TAK ADA ORANG','TIADA ORANG PAGI','MALAM SAHAJA ADA','MALAM SAJA','SIANG BEKERJA','SIANG HARI','WAKTU SIANG','PREMIS KOSONG','KOSONG','TIADA ORANG','JARANG GUNA']



In [195]:
df_no['keywords'] = [tiada_orang_keywords for _ in range(len(df_no))]

In [196]:
df_no['tiada_orang_keywords_flag'] = df_no.apply(search_func, axis=1)

In [197]:
df_no.tiada_orang_keywords_flag.value_counts()

0    3122
1       2
Name: tiada_orang_keywords_flag, dtype: int64

- u can simply add new keywords just append the new keyword in below set of list.

In [198]:
not_resolved_keywords = ['SERVICE ORDER', 'SERVICE NOTIFICATION','HAS BEEN CREATED', 'SO HAS ALREADY TECOED','NEW SERVICE NOTIFICATION']

In [199]:
df_no['keywords'] = [not_resolved_keywords for _ in range(len(df_no))]

In [200]:
df_no['not_resolved_flag'] = df_no.apply(search_func, axis=1)

In [201]:
df_no.not_resolved_flag.value_counts()

0    3119
1       5
Name: not_resolved_flag, dtype: int64

In [202]:
#delete the columns from dataframe 
del df_no['keywords'] 
del df_no['clean_sn_remark_grams']


In [203]:
# #create sum column for all the flags
df_no['flag_sum'] = df_no['no_anomaly_customer_consumption_lifestyle_festive_keywords_flag'] +  df_no['smart_meter_keywords_flag'] + df_no['no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag'] +df_no['electrical_within_usage_keywords_flag'] +df_no['weather_keywords_flag']  + df_no['move_out_keywords_flag'] + df_no['tiada_orang_keywords_flag'] + df_no['not_resolved_flag']

Let's look at the dataframe 

In [204]:
df_no.head()

Complaint_ID                 Category      Sub_Category  \
0    1000684273  OPC Meter Accuracy Test  High Bill Normal   
1    1000697304  OPC Meter Accuracy Test  High Bill Normal   
2    1000678345  OPC Meter Accuracy Test  High Bill Normal   
3    1000703326  OPC Meter Accuracy Test  High Bill Normal   
4    1000692865  OPC Meter Accuracy Test  High Bill Normal   

                                   SN_Remarks_31July     Verified_Root_Cause  \
0   Internal note 07.06.2019 15:12:21 10077213  p...                METER OK   
1   Internal note 13.06.2019 15:46:13 10076230  p...                METER OK   
2   Internal note 03.06.2019 11:14:39 10091193  m...                METER OK   
3   Internal note 17.06.2019 10:39:55 60068218 - ...                METER OK   
4   Internal note 12.06.2019 11:52:38 10088233  m...  WITHIN PREV PEAK RANGE   

                                    clean_sn_remarks  \
0  INTERNAL NOTE PENGGUNA MAHU PIHAK TNB SEMAK KE...   
1  INTERNAL NOTE PENGGUNA MOHON TEST METER CALL P...   
2  INTERNAL NOTE MOHON SEMAKAN KEJITUAN METER PEN...   
3  INTERNAL NOTE PENGGUNA MAKLUM BIL MENINGKAT TI...   
4  INTERNAL NOTE MOHON SEMAKAN METER PENGGUNA MAK...   

   no_anomaly_customer_consumption_lifestyle_festive_keywords_flag  \
0                                                  0                 
1                                                  0                 
2                                                  0                 
3                                                  0                 
4                                                  0                 

   smart_meter_keywords_flag  \
0                          0   
1                          0   
2                          0   
3                          0   
4                          0   

   no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag  \
0                                                  0                        
1                                                  0                        
2                                                  0                        
3                                                  0                        
4                                                  0                        

   electrical_within_usage_keywords_flag  weather_keywords_flag  \
0                                      1                      0   
1                                      1                      0   
2                                      1                      0   
3                                      1                      0   
4                                      1                      0   

   move_out_keywords_flag  tiada_orang_keywords_flag  not_resolved_flag  \
0                       0                          0                  0   
1                       0                          0                  0   
2                       0                          0                  0   
3                       0                          0                  0   
4                       0                          0                  0   

   flag_sum  
0         1  
1         1  
2         1  
3         1  
4         1

In [205]:
df_no.columns


Index(['Complaint_ID', 'Category', 'Sub_Category', 'SN_Remarks_31July',
       'Verified_Root_Cause', 'clean_sn_remarks',
       'no_anomaly_customer_consumption_lifestyle_festive_keywords_flag',
       'smart_meter_keywords_flag',
       'no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag',
       'electrical_within_usage_keywords_flag', 'weather_keywords_flag',
       'move_out_keywords_flag', 'tiada_orang_keywords_flag',
       'not_resolved_flag', 'flag_sum'],
      dtype='object')

In [206]:
#please mention only the columns name with flag
cols = ['no_anomaly_customer_consumption_lifestyle_festive_keywords_flag',
       'smart_meter_keywords_flag',
       'no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag',
       'electrical_within_usage_keywords_flag', 'weather_keywords_flag',
       'move_out_keywords_flag', 'tiada_orang_keywords_flag',
       'not_resolved_flag']
df_no[cols] = df_no[cols].replace({0.:0, 1.:1}) # to assigned value 

In [207]:
#do the same things, to mentionthe columns name with flag
#create new column named possible root cause 
df_no['possible_root_cause'] = df_no[list(['no_anomaly_customer_consumption_lifestyle_festive_keywords_flag',
       'smart_meter_keywords_flag',
       'no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag',
       'electrical_within_usage_keywords_flag', 'weather_keywords_flag',
       'move_out_keywords_flag', 'tiada_orang_keywords_flag',
       'not_resolved_flag'])].eq(1, axis=0).apply(lambda x: ','.join(x[x].index), axis=1)

Ok, so we need to mention the all the columns name for below cell. 


In [208]:
df_no = df_no[['Complaint_ID', 'Category', 'Sub_Category', 'SN_Remarks_31July',
       'Verified_Root_Cause', 'clean_sn_remarks',
       'no_anomaly_customer_consumption_lifestyle_festive_keywords_flag',
       'smart_meter_keywords_flag',
       'no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag',
       'electrical_within_usage_keywords_flag', 'weather_keywords_flag',
       'move_out_keywords_flag', 'tiada_orang_keywords_flag',
       'not_resolved_flag', 'flag_sum', 'possible_root_cause']]

lets check the possible root cause 

In [209]:
df_no.possible_root_cause.value_counts()

                                                                                          3066
electrical_within_usage_keywords_flag                                                       51
electrical_within_usage_keywords_flag,not_resolved_flag                                      4
electrical_within_usage_keywords_flag,tiada_orang_keywords_flag                              1
not_resolved_flag                                                                            1
electrical_within_usage_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag       1
Name: possible_root_cause, dtype: int64

In [210]:
df_no['possible_root_cause'].astype('category').value_counts()

                                                                                          3066
electrical_within_usage_keywords_flag                                                       51
electrical_within_usage_keywords_flag,not_resolved_flag                                      4
not_resolved_flag                                                                            1
electrical_within_usage_keywords_flag,tiada_orang_keywords_flag                              1
electrical_within_usage_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag       1
Name: possible_root_cause, dtype: int64

if its contained multiple root cause, please execute below command to pick the dominant root cause

In [211]:
#Replace whole string if it contains substring
df_no.loc[df_no['possible_root_cause'].str.contains('not_resolved_flag'), 'possible_root_cause'] = 'not_resolved_flag'

In [212]:
df_no['possible_root_cause']  = df_no['possible_root_cause'].replace({"electrical_within_usage_keywords_flag,tiada_orang_keywords_flag": "electrical_within_usage_keywords_flag", "smart_meter_keywords_flag,electrical_within_usage_keywords_flag": "smart_meter_keywords_flag","electrical_within_usage_keywords_flag,weather_keywords_flag":"electrical_within_usage_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,electrical_within_usage_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","electrical_within_usage_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag":"move_out_keywords_flag","electrical_within_usage_keywords_flag,move_out_keywords_flag":"move_out_keywords_flag","move_out_keywords_flag,tiada_orang_keywords_flag":"move_out_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,electrical_within_usage_keywords_flag,weather_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","electrical_within_usage_keywords_flag,weather_keywords_flag,tiada_orang_keywords_flag":"electrical_within_usage_keywords_flag","smart_meter_keywords_flag,tiada_orang_keywords_flag":"smart_meter_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,weather_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,electrical_within_usage_keywords_flag,tiada_orang_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,electrical_within_usage_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","smart_meter_keywords_flag,electrical_within_usage_keywords_flag,tiada_orang_keywords_flag":"smart_meter_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","electrical_within_usage_keywords_flag,weather_keywords_flag,move_out_keywords_flag":"electrical_within_usage_keywords_flag","weather_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag":"move_out_keywords_flag","no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,electrical_within_usage_keywords_flag":"no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,electrical_within_usage_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,electrical_within_usage_keywords_flag,weather_keywords_flag":"no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag","electrical_within_usage_keywords_flag,not_resolved_flag":"not_resolved_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,electrical_within_usage_keywords_flag,weather_keywords_flag,tiada_orang_keywords_flag":"electrical_within_usage_keywords_flag"})

In [213]:
df_no['possible_root_cause']  = df_no['possible_root_cause'].replace({"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,electrical_within_usage_keywords_flag,weather_keywords_flag":"no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag","no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag":"no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,weather_keywords_flag":"no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag","no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,weather_keywords_flag":"no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,tiada_orang_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,electrical_within_usage_keywords_flag,tiada_orang_keywords_flag":"tiada_orang_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,electrical_within_usage_keywords_flag,weather_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,electrical_within_usage_keywords_flag,weather_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag":"tiada_orang_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,electrical_within_usage_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag":"tiada_orang_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,weather_keywords_flag,tiada_orang_keywords_flag":"tiada_orang_keywords_flag","no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,electrical_within_usage_keywords_flag,move_out_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","electrical_within_usage_keywords_flag,weather_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag":"tiada_orang_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,smart_meter_keywords_flag,electrical_within_usage_keywords_flag":"smart_meter_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag,electrical_within_usage_keywords_flag,tiada_orang_keywords_flag":"no_anomaly_customer_consumption_lifestyle_festive_keywords_flag","no_anomaly_customer_consumption_lifestyle_festive_keywords_flag,electrical_within_usage_keywords_flag,move_out_keywords_flag,tiada_orang_keywords_flag":"tiada_orang_keywords_flag"}) 

In [214]:
df_no['possible_root_cause'].value_counts()

                                         3066
electrical_within_usage_keywords_flag      52
not_resolved_flag                           5
move_out_keywords_flag                      1
Name: possible_root_cause, dtype: int64

#### extracting so/sn if necessary 

In [215]:
filter_so_sn = df_no.ix[df_no['possible_root_cause'] == "not_resolved_flag"]
filter_so_sn.to_csv(r"C:\Users\ijohan\Desktop\backup\izzat_tnb\so_sn_July_data_run_on_25July.csv")

C:\Users\ijohan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [216]:
filter_so_sn['possible_root_cause'].value_counts()

not_resolved_flag    5
Name: possible_root_cause, dtype: int64

In [217]:
#removing the complaint_id in filter_so_sn dataframe in the main dataframe df_no
cond = df_no['Complaint_ID'].isin(filter_so_sn['Complaint_ID']) == True
df_no.drop(df_no[cond].index, inplace = True)

In [218]:
df_no["possible_root_cause"].value_counts()

                                         3066
electrical_within_usage_keywords_flag      52
move_out_keywords_flag                      1
Name: possible_root_cause, dtype: int64

In [219]:
sn_remarks_flagged = df_no[df_no['flag_sum'] != 0]
sn_remarks_further = df_no[df_no['flag_sum'] == 0]

In [64]:
# if you found any error to save dataframe to your current path, please execute this command
#for col in sn_remarks_further.columns:
#    if sn_remarks_further[col].dtype==object:
#        sn_remarks_further[col]=sn_remarks_further[col].apply(lambda x: np.nan if x==np.nan else str(x).encode('utf-8', 'replace').decode('utf-8'))

C:\Users\ijohan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [221]:
sn_remarks_further.head()

Complaint_ID                 Category  \
6     1000701341  OPC Meter Accuracy Test   
20    1000715387    Request to Test Meter   
21    1000699960         Bilik Gerakan SM   
22    1000679836    Request to Test Meter   
23    1000724708         Bilik Gerakan SM   

                                     Sub_Category  \
6                                High Bill Normal   
20                                   Not assigned   
21  High / Low Bill - Normal Bill 3 Month & Above   
22                                   Not assigned   
23  High / Low Bill - Normal Bill 3 Month & Above   

                                    SN_Remarks_31July  \
6    Internal note 15.06.2019 11:55:42 10046553  L...   
20                                                NaN   
21                                                NaN   
22                                                NaN   
23                                                NaN   

        Verified_Root_Cause  \
6   SEASONAL CONS BEHAVIOUR   
20  SEASONAL CONS BEHAVIOUR   
21  SEASONAL CONS BEHAVIOUR   
22  SEASONAL CONS BEHAVIOUR   
23  SEASONAL CONS BEHAVIOUR   

                                     clean_sn_remarks  \
6   INTERNAL NOTE LAPORAN MRE FAZLI BACAAN NORMAL ...   
20                                                NAN   
21                                                NAN   
22                                                NAN   
23                                                NAN   

    no_anomaly_customer_consumption_lifestyle_festive_keywords_flag  \
6                                                   0                 
20                                                  0                 
21                                                  0                 
22                                                  0                 
23                                                  0                 

    smart_meter_keywords_flag  \
6                           0   
20                          0   
21                          0   
22                          0   
23                          0   

    no_anomaly_customer_consumption_lifestyle_school_holiday_keywords_flag  \
6                                                   0                        
20                                                  0                        
21                                                  0                        
22                                                  0                        
23                                                  0                        

    electrical_within_usage_keywords_flag  weather_keywords_flag  \
6                                       0                      0   
20                                      0                      0   
21                                      0                      0   
22                                      0                      0   
23                                      0                      0   

    move_out_keywords_flag  tiada_orang_keywords_flag  not_resolved_flag  \
6                        0                          0                  0   
20                       0                          0                  0   
21                       0                          0                  0   
22                       0                          0                  0   
23                       0                          0                  0   

    flag_sum possible_root_cause  
6          0                      
20         0                      
21         0                      
22         0                      
23         0

save dataframe flagged and unflagged to your path.

In [145]:
sn_remarks_flagged.to_csv(r"C:\Users\ijohan\Desktop\analysis\june\31072019\FLAGGED_JUNE_NO_ANOMALY_DATA_344PM.csv")
sn_remarks_further.to_csv(r"C:\Users\ijohan\Desktop\analysis\june\31072019\UNFLAGGED_JUNE_NO_ANOMALY_DATA_344PM.csv")